In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_community.utilities.sql_database import SQLDatabase
# from langchain_core.runnables import RunnableSequence
import requests
import os

from langchain.tools import BaseTool
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableMap, RunnableSequence

from dotenv import load_dotenv, find_dotenv
import sys

sys.path.insert(0, os.path.abspath("injections"))


from Plunger_helper import raw_json

schema_context = raw_json["PromptInjection"]

load_dotenv(find_dotenv())  
# --- LLM Setup ---
from langchain_groq import ChatGroq


llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)



In [2]:
import re

In [3]:
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langchain.tools import BaseTool
from typing import TypedDict
import requests
import json


# ===== 1. STATE =====
class ChatState(TypedDict):
    query: str
    structured_query: str
    sql: str
    result: str
    answer: str


# ===== 1. TOOL =====
class ExecuteSQLTool(BaseTool):
    name: str = "execute_sql"
    description: str = "Executes a SELECT/WITH SQL query on the live plunger lift API and returns JSON."

    def _run(self, query: str) -> str:
        query = self._extract_sql(query)
        if not self._is_select_or_with(query):
            return "Query rejected: Only SELECT/WITH statements allowed."

        try:
            resp = requests.post(
                "http://localhost:8765/",
                headers={"Content-Type": "text/plain"},
                data=query
            )
            if resp.status_code != 200:
                return f"Error {resp.status_code}: {resp.text}"
            return resp.text
        except Exception as e:
            return f"Request failed: {str(e)}"

    async def _arun(self, query: str) -> str:
        raise NotImplementedError

    def _extract_sql(self, text: str) -> str:
        match = re.search(r"(?is)\b(SELECT|WITH)\b.*", text)
        if not match:
            return text.strip()
        sql = match.group(0).strip()
        # Stop at first semicolon (ignore everything after)
        sql = sql.split(";")[0]
        return sql


    def _is_select_or_with(self, sql: str) -> bool:
        return re.match(r"(?is)^\s*(SELECT|WITH)\b", sql) is not None


execute_sql_tool = ExecuteSQLTool()

# ===== 2. PROMPTS =====
decomposer_prompt = ChatPromptTemplate.from_template("""
You are a task decomposer. Break down the user's natural language question into a structured query that highlights what the user wants to know, what metrics or filters are involved, and what kind of result is expected.
## User Query:
{user_query}
Return a structured breakdown only.
""")

sql_prompt = ChatPromptTemplate.from_template("""
You are an expert SQL generator. Given a structured query and schema context, return a valid SQL query for an SQLite database. Do not explain anything.

""")

summary_prompt = ChatPromptTemplate.from_template("""
You are a data analyst. Based on the following SQL query, its result, and the user’s original question, generate a clear and concise explanation that helps a human understand the insight in plain English.

Question: {query}
SQL: {sql}
Result: {result}
""")

# ===== 3. LANGGRAPH =====
def build_graph(llm, schema_context):
    g = StateGraph(dict)

    # Step 1: Decompose
    def decompose(state):
        structured = (decomposer_prompt | llm | StrOutputParser()).invoke({"query": state["query"]})
        return {**state, "structured_query": structured}

    # Step 2: SQL generation
    def gen_sql(state):
        sql_text = (sql_prompt | llm | StrOutputParser()).invoke({
            "schema": schema_context,
            "structured_query": state["structured_query"]
        })
        clean_sql = execute_sql_tool._extract_sql(sql_text)
        return {**state, "sql": clean_sql}

    # Step 3: Execute SQL
    def exec_sql(state):
        result = execute_sql_tool.run(state["sql"])
        return {**state, "result": result}

    # Step 4: Summarize
    def summarize(state):
        summary = (summary_prompt | llm | StrOutputParser()).invoke({
            "query": state["query"],
            "sql": state["sql"],
            "result": state["result"]
        })
        return {**state, "final_answer": summary}

    g.add_node("decompose", decompose)
    g.add_node("gen_sql", gen_sql)
    g.add_node("exec_sql", exec_sql)
    g.add_node("summarize", summarize)

    g.add_edge("decompose", "gen_sql")
    g.add_edge("gen_sql", "exec_sql")
    g.add_edge("exec_sql", "summarize")
    g.add_edge("summarize", END)

    g.set_entry_point("decompose")
    return g.compile()

# ===== 4. RUN =====
if __name__ == "__main__":

    graph = build_graph(llm, schema_context)
    user_q = input("Enter your plunger lift question: ")
    result = graph.invoke({"query": user_q})
    print("\nFinal Answer:\n", result["final_answer"])


KeyError: "Input to ChatPromptTemplate is missing variables {'user_query'}.  Expected: ['user_query'] Received: ['query']\nNote: if you intended {user_query} to be part of the string and not a variable, please escape it with double curly braces like: '{{user_query}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "